In [1]:
import os
import datetime
import random

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# sklearn
from sklearn.model_selection import train_test_split
from utils import get_merged_df

In [2]:
# tensorflow
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

import keras
from keras.models import Sequential, Model  # V2 is tensorflow.keras.xxxx, V1 is keras.xxx
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, Input, GlobalAveragePooling2D
from keras.models import load_model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.metrics import F1Score, AUC, CategoricalAccuracy, BinaryAccuracy
from tensorflow.keras.optimizers import RMSprop

print( f'tf.__version__: {tf.__version__}' )
print( f'keras.__version__: {keras.__version__}' )


tf.__version__: 2.15.0
keras.__version__: 2.15.0


In [3]:
import cv2
from PIL import Image

In [4]:
data_dir = 'training_data/training_data'
norm_csv_path = 'training_data/training_norm.csv'
cleaned_df = get_merged_df(data_dir, norm_csv_path)

len(cleaned_df)

In [ ]:
# angle_labels = cleaned_df['angle'].to_list()
# speed_labels = cleaned_df['speed'].to_list()
# image_paths = cleaned_df['image_path'].to_list()

X_train, X_valid, y_train, y_valid = train_test_split(cleaned_df['image_path'].to_list(), cleaned_df['angle'].to_list(), test_size=0.3)

# X_train, X_valid, angle_train, angle_valid, speed_train, speed_valid = train_test_split(image_paths, angle_labels, speed_labels, test_size=0.3)
print("Training data: %d\nValidation data: %d" % (len(X_train), len(X_valid)))

# print(type(angle_labels))

Training data: 9654
Validation data: 4138


In [ ]:
def my_imread(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def img_preprocess(image):
    # height, _, _ = image.shape
    # image = image[int(height/2):,:,:]  # remove top half of the image, as it is not relavant for lane following
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)  # Nvidia model said it is best to use YUV color space
    # image = cv2.GaussianBlur(image, (3,3), 0)
    image = cv2.resize(image, (224,224)) # input image size (200,66) Nvidia model
    # image = image / 255 # normalizing, the processed image becomes black for some reason.  do we need this?
    # image = (image - 127.5) / 127.5
    return image


# fig, axes = plt.subplots(1, 2, figsize=(15, 10))
# image_orig = my_imread(merged_df['image_path'][image_index])
# image_processed = img_preprocess(image_orig)
# axes[0].imshow(image_orig)
# axes[0].set_title("orig")
# axes[1].imshow(image_processed)
# axes[1].set_title("processed")

In [ ]:
def image_data_generator(image_paths, angle_labels, batch_size):
    while True:
        batch_images = []
        batch_angles = []

        for i in range(batch_size):
            random_index = random.randint(0, len(image_paths) - 1)
            image = my_imread(image_paths[random_index])
            angle_label = angle_labels[random_index]
            angle_label *= 16

            image = img_preprocess(image)
            batch_images.append(image)

            angle_one_hot = to_categorical(angle_label, num_classes=17)
            batch_angles.append(angle_one_hot)

        yield( np.asarray(batch_images), np.asarray(batch_angles))

In [ ]:
def mobile_net_classification_model():
    inputs = Input(shape=(224, 224, 3))
    x = tf.keras.layers.Rescaling(1./127.5, offset=-1)(inputs)

    base_model = tf.keras.applications.MobileNetV2(include_top=False, weights="imagenet", input_tensor=x)
    base_model.trainable = False

    x = GlobalAveragePooling2D()(base_model.output)

    # Common part of the model
    common = Dense(1024, activation='relu')(x)
    common = Dropout(0.3)(common)

    # Branch for the angle prediction (multi-class classification)
    angle_branch = Dense(512, activation='relu')(common)
    angle_branch = Dropout(0.3)(angle_branch)
    angle_output = Dense(17, activation='softmax', name='angle_output')(angle_branch) # 17 classes for angle

    model = Model(inputs=inputs, outputs=angle_output)

    # Create an RMSprop optimizer with a custom learning rate
    custom_lr = 0.01  # Example custom learning rate
    optimizer = RMSprop(learning_rate=custom_lr)

    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics='accuracy')

    return base_model, model

# model = nvidia_model()
base_model, model = mobile_net_classification_model()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_1 (Rescaling)     (None, 224, 224, 3)          0         ['input_2[0][0]']             
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['rescaling_1[0][0]']         
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                        

In [ ]:
model_output_dir = 'models/angle'

# start Tensorboard before model fit, so we can see the epoch tick in Tensorboard
# Jupyter Notebook embedded Tensorboard is a new feature in TF 2.0!!  

# clean up log folder for tensorboard
log_dir_root = f'{model_output_dir}/logs'
#!rm -rf $log_dir_root

tensorboard_callback = TensorBoard(log_dir_root, histogram_freq=1)

# Specify the file path where you want to save the model
filepath = 'models/angle/{epoch:02d}-{val_loss:.2f}'

# Create the ModelCheckpoint callback
model_checkpoint_callback = ModelCheckpoint(
    filepath,
    monitor='val_loss',     # Monitor validation loss
    verbose=1,              # Log a message each time the callback saves the model
    save_best_only=True,    # Only save the model if 'val_loss' has improved
    save_weights_only=False, # Only save the weights of the model
    mode='min',             # 'min' means the monitored quantity should decrease
    save_freq='epoch'       # Check every epoch
)

In [ ]:
history = model.fit(
    image_data_generator(X_train, y_train, batch_size=200),
    steps_per_epoch=100,
    epochs=10,
    validation_data = image_data_generator(X_valid, y_valid, batch_size=200),
    verbose=1,
    shuffle=1,
    callbacks=[model_checkpoint_callback, tensorboard_callback]
)

Epoch 1/10
 12/100 [==>...........................] - ETA: 2:42 - loss: 4.0060 - accuracy: 0.1946